In [ ]:
import requests
import pandas as pd
from datetime import datetime

# Helper to convert epoch millis → YYYY-MM-DD HH:MM
def convert_ts(ts):
    return datetime.utcfromtimestamp(int(ts) / 1000).strftime("%Y-%m-%d %H:%M") if ts else None

# Fetch API data
def fetch_api(url, headers=None, params=None):
    if headers is None:
        headers = {
	        "x-rapidapi-key": "fa8e2f141fmsh93e60c107630b1ep15131djsnd985ef01f9ec",
	        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Helper: Format innings scores
def format_scores(team_score):
    if not team_score:
        return ""
    scores = []
    for key, innings in team_score.items():
        if innings:
            runs = innings.get("runs", 0)
            wickets = innings.get("wickets", 0)
            overs = innings.get("overs", 0)
            scores.append(f"{runs}/{wickets} ({overs} ov)")
    return " | ".join(scores)

# Extract recent matches
def extract_recent_matches(data):
    all_matches = []
    type_matches = data.get("typeMatches", [])

    for t in type_matches:
        series_matches = t.get("seriesMatches", [])
        for s in series_matches:
            series_ad_wrapper = s.get("seriesAdWrapper", {})
            matches = series_ad_wrapper.get("matches", [])
            for m in matches:
                info = m.get("matchInfo", {})
                score = m.get("matchScore", {})

                team1_score = format_scores(score.get("team1Score"))
                team2_score = format_scores(score.get("team2Score"))

                start_dt = datetime.fromtimestamp(int(info.get("startDate", 0))/1000)

                match_data = {
                    "match_id": info.get("matchId"),
                    "match_desc": info.get("matchDesc"),
                    "team1": info.get("team1", {}).get("teamName"),
                    "team2": info.get("team2", {}).get("teamName"),
                    "start_date": start_dt,
                    "status": info.get("status"),
                    "state": info.get("state"),
                    "venue": info.get("venueInfo", {}).get("ground"),
                    "venue_city": info.get("venueInfo", {}).get("city"),
                    "series_name": info.get("seriesName"),
                    "match_format": info.get("matchFormat"),
                    "team1_score": team1_score,
                    "team2_score": team2_score
                }
                all_matches.append(match_data)

    return pd.DataFrame(all_matches)

# Call for recent matches
url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
data = fetch_api(url)
df_recent_matches = extract_recent_matches(data)

print("Recent Matches DataFrame:")
df_recent_matches


Recent Matches DataFrame:


,match_id,match_desc,team1,team2,start_date,status,state,venue,venue_city,series_name,match_format,team1_score,team2_score
0,130129,"Super Fours, 14th Match (A1 v A2)",Pakistan,India,2025-09-21 20:00:00,India won by 6 wkts,Complete,Dubai International Cricket Stadium,Dubai,Asia Cup 2025,T20,171/5 (19.6 ov),174/4 (18.5 ov)
1,130096,"10th Match, Group A",Pakistan,United Arab Emirates,2025-09-17 20:00:00,Pakistan won by 41 runs,Complete,Dubai International Cricket Stadium,Dubai,Asia Cup 2025,T20,146/9 (19.6 ov),105/10 (17.4 ov)
2,130085,"9th Match, Group B",Bangladesh,Afghanistan,2025-09-16 20:00:00,Bangladesh won by 8 runs,Complete,Sheikh Zayed Stadium,Abu Dhabi,Asia Cup 2025,T20,154/5 (19.6 ov),146/10 (19.6 ov)
3,130124,"Super Fours, 13th Match (B1 v B2)",Sri Lanka,Bangladesh,2025-09-20 20:00:00,Bangladesh won by 4 wkts,Complete,Dubai International Cricket Stadium,Dubai,Asia Cup 2025,T20,168/7 (19.6 ov),169/6 (19.5 ov)
4,130074,"7th Match, Group A",United Arab Emirates,Oman,2025-09-15 17:30:00,United Arab Emirates won by 42 runs,Complete,Sheikh Zayed Stadium,Abu Dhabi,Asia Cup 2025,T20,172/5 (19.6 ov),130/10 (18.4 ov)
5,130113,"12th Match, Group A",India,Oman,2025-09-19 20:00:00,India won by 21 runs,Complete,Sheikh Zayed Stadium,Abu Dhabi,Asia Cup 2025,T20,188/8 (19.6 ov),167/4 (19.6 ov)
6,130102,"11th Match, Group B",Afghanistan,Sri Lanka,2025-09-18 20:00:00,Sri Lanka won by 6 wkts,Complete,Sheikh Zayed Stadium,Abu Dhabi,Asia Cup 2025,T20,169/8 (19.6 ov),171/4 (18.4 ov)
7,130080,"8th Match, Group B",Hong Kong,Sri Lanka,2025-09-15 20:00:00,Sri Lanka won by 4 wkts,Complete,Dubai International Cricket Stadium,Dubai,Asia Cup 2025,T20,149/4 (19.6 ov),153/6 (18.5 ov)
8,105850,2nd T20I,Ireland,England,2025-09-19 18:00:00,Match abandoned due to rain (no toss),Abandon,The Village,Dublin,"England tour of Ireland, 2025",T20,,
9,105858,3rd T20I,Ireland,England,2025-09-21 18:00:00,England won by 6 wkts,Complete,The Village,Dublin,"England tour of Ireland, 2025",T20,154/8 (19.6 ov),155/4 (17.1 ov)


In [64]:
# Step 1: Import SQLAlchemy
from sqlalchemy import create_engine, text

# Step 2: Function to create MySQL engine
def create_mysql_engine(username, password, host, database):
    """
    Creates and returns a SQLAlchemy MySQL engine.
    """
    return create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")

# Step 3: Function to create table if it does not exist
def create_recent_matches_table(engine):
    """
    Creates 'recent_matches' table in MySQL if it doesn't already exist.
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS recent_matches (
        match_id BIGINT PRIMARY KEY,
        match_desc VARCHAR(100),
        team1 VARCHAR(50),
        team2 VARCHAR(50),
        start_date DATETIME,
        status VARCHAR(100),
        state VARCHAR(50),
        venue VARCHAR(100),
        venue_city VARCHAR(100),
        series_name VARCHAR(150),
        match_format VARCHAR(20),
        team1_score VARCHAR(50),
        team2_score VARCHAR(50)
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'recent_matches' is ready.")

# Step 4: Function to insert DataFrame into MySQL
def insert_df_to_mysql(df, engine, table_name):
    """
    Inserts a DataFrame into a specified MySQL table.
    """
    # Replace NaN with None to avoid issues
    df = df.where(pd.notnull(df), None)

    # Append data to table
    df.to_sql(table_name, con=engine, if_exists="replace", index=False)
    print(f"Data inserted successfully into '{table_name}'!")

# Step 5: Connect, create table, and insert data
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_recent_matches_table(engine)
insert_df_to_mysql(df_recent_matches, engine, table_name="recent_matches")


Table 'recent_matches' is ready.
Data inserted successfully into 'recent_matches'!


In [4]:
import requests
import pandas as pd

# Extract stat for one format
def extract_stat(data, stat_name, format_type):
    headers = data.get("headers", [])
    values_list = data.get("values", [])
    
    if format_type not in headers:
        return 0
    
    col_index = headers.index(format_type)
    
    for item in values_list:
        row = item.get("values", [])
        if row and row[0].lower() == stat_name.lower():
            value = row[col_index]
            try:
                return int(value)
            except (ValueError, TypeError):
                return 0
    return 0

# Sum stats across formats
def sum_stats_across_formats(data, stat_name):
    # Use exact headers from API: Test, ODI, T20, IPL
    formats = ["Test", "ODI", "T20"]
    total = 0
    for fmt in formats:
        total += extract_stat(data, stat_name, fmt)
    return total

# Fetch player summary
def fetch_players_summary(player_ids):
    all_players = []
    for player_id in player_ids:
        try:
            # Player general info
            player_info = fetch_api(f"https://cricbuzz-cricket2.p.rapidapi.com/stats/v1/player/{player_id}")
            # Batting stats
            batting_info = fetch_api(f"https://cricbuzz-cricket2.p.rapidapi.com/stats/v1/player/{player_id}/batting")
            # Bowling stats
            bowling_info = fetch_api(f"https://cricbuzz-cricket2.p.rapidapi.com/stats/v1/player/{player_id}/bowling")
            
            player_data = {
                "player_id": player_id,
                "full_name": player_info.get("name"),
                "name": player_info.get("nickName"),
                "country": player_info.get("intlTeam"),
                "playing_role": player_info.get("role"),
                "batting_style": player_info.get("bat"),
                "bowling_style": player_info.get("bowl"),
                "total_runs": sum_stats_across_formats(batting_info, "Runs"),
                "total_wickets": sum_stats_across_formats(bowling_info, "Wickets")
            }
            
            all_players.append(player_data)
            print(f" Player {player_id} data fetched")
        except Exception as e:
            print(f" Error fetching player {player_id}: {e}")
    
    return all_players

# Player IDs to fetch
player_ids = [576, 25, 265, 1413, 11808, 8733, 10808, 13217, 10551, 9311, 587, 8808, 9647, 10276, 10744, 8271,11540, 18504, 9720, 10738, 9551, 11177,
               10100, 8313]  

# Fetch data & convert to DataFrame
players_summary = fetch_players_summary(player_ids)
df_players = pd.DataFrame(players_summary)

# Display final DataFrame
print("Final Player Summary DataFrame:")
df_players


 Player 576 data fetched
 Player 25 data fetched
 Player 265 data fetched
 Player 1413 data fetched
 Player 11808 data fetched
 Player 8733 data fetched
 Player 10808 data fetched
 Player 13217 data fetched
 Player 10551 data fetched
 Player 9311 data fetched
 Player 587 data fetched
 Player 8808 data fetched
 Player 9647 data fetched
 Player 10276 data fetched
 Player 10744 data fetched
 Player 8271 data fetched
 Player 11540 data fetched
 Player 18504 data fetched
 Player 9720 data fetched
 Player 10738 data fetched
 Player 9551 data fetched
 Player 11177 data fetched
 Player 10100 data fetched
 Player 8313 data fetched
Final Player Summary DataFrame:


,player_id,full_name,name,country,playing_role,batting_style,bowling_style,total_runs,total_wickets
0,576,Rohit Sharma,Rohit,India,Batsman,Right Handed Bat,Right-arm offbreak,19700,12
1,25,Sachin Tendulkar,Tendulkar,India,Batsman,Right Handed Bat,Right-arm legbreak,34357,201
2,265,MS Dhoni,Dhoni,India,WK-Batsman,Right Handed Bat,Right-arm medium,17266,1
3,1413,Virat Kohli,Kohli,India,Batsman,Right Handed Bat,Right-arm medium,27599,9
4,11808,Shubman Gill,Gill,India,Batsman,Right Handed Bat,Right-arm offbreak,6082,0
5,8733,KL Rahul,Rahul,India,WK-Batsman,Right Handed Bat,None,9097,0
6,10808,Mohammed Siraj,Siraj,India,Bowler,Right Handed Bat,Right-arm fast,220,208
7,13217,Arshdeep Singh,Arshdeep Singh,India,Bowler,Left Handed Bat,Left-arm fast-medium,111,114
8,10551,Prasidh Krishna,Prasidh,India,Bowler,Right Handed Bat,Right-arm fast,12,59
9,9311,Jasprit Bumrah,Bumrah,India,Bowler,Right Handed Bat,Right-arm fast,440,460


In [5]:
# Function to create table if it does not exist
def create_players_table(engine):
    """
    Creates 'players' table in MySQL if it doesn't already exist.
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS players (
        player_id INT PRIMARY KEY,
        full_name VARCHAR(100),
        name VARCHAR(150),
        country VARCHAR(50),
        playing_role VARCHAR(50),
        batting_style VARCHAR(50),
        bowling_style VARCHAR(50),
        total_runs INT,
        total_wickets INT
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'players' is ready.")

# Connect, create table, and insert data
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_players_table(engine)
insert_df_to_mysql(df_players, engine, table_name="players")

Table 'players' is ready.
Data inserted successfully into 'players'!


In [70]:
# Helper: Format innings scores
def format_scores(team_score):
    if not team_score:
        return ""
    scores = []
    for key, innings in team_score.items():
        if innings:
            runs = innings.get("runs", 0)
            wickets = innings.get("wickets", 0)
            overs = innings.get("overs", 0)
            scores.append(f"{runs}/{wickets} ({overs} ov)")
    return " | ".join(scores)

# Extract match details from one series
def extract_series_matches(data):
    all_matches = []
    match_details = data.get("matchDetails", [])

    for item in match_details:
        map_data = item.get("matchDetailsMap")
        if not map_data:
            continue

        matches = map_data.get("match", [])
        for m in matches:
            info = m.get("matchInfo", {})
            score = m.get("matchScore", {})

            team1_score = format_scores(score.get("team1Score"))
            team2_score = format_scores(score.get("team2Score"))

            start_dt = datetime.fromtimestamp(int(info.get("startDate", 0))/1000)
            end_dt = datetime.fromtimestamp(int(info.get("endDate", 0))/1000)

            match_data = {
                "match_id": info.get("matchId"),
                "match_desc": info.get("matchDesc"),
                "team1": info.get("team1", {}).get("teamName"),
                "team2": info.get("team2", {}).get("teamName"),
                "start_date": start_dt,
                "end_date": end_dt,
                "status": info.get("status"),
                "state": info.get("state"),
                "venue": info.get("venueInfo", {}).get("ground"),
                "series_name": info.get("seriesName"),
                "match_format": info.get("matchFormat"),
                "team1_score": team1_score,
                "team2_score": team2_score
            }
            all_matches.append(match_data)

    return pd.DataFrame(all_matches)

# Fetch multiple series
series_ids = [8393, 4905, 8528, 3692]  
all_matches_df = pd.DataFrame()

for sid in series_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{sid}"
    data = fetch_api(url)
    df = extract_series_matches(data)
    all_matches_df = pd.concat([all_matches_df, df], ignore_index=True)

# Display final combined DataFrame
print("All Series Matches DataFrame:")
all_matches_df


All Series Matches DataFrame:


,match_id,match_desc,team1,team2,start_date,end_date,status,state,venue,series_name,match_format,team1_score,team2_score
0,100220,1st Test,INDIA,BANGLADESH,2024-09-19 09:30:00,2024-09-23 09:30:00,India won by 280 runs,complete,MA Chidambaram Stadium,"Bangladesh tour of India, 2024",TEST,376/10 (91.2 ov) | 287/4 (64.0 ov),149/10 (47.1 ov) | 234/10 (62.1 ov)
1,100229,2nd Test,INDIA,BANGLADESH,2024-09-27 09:30:00,2024-10-01 09:30:00,India won by 7 wkts,complete,Green Park,"Bangladesh tour of India, 2024",TEST,285/9 (34.4 ov) | 98/3 (17.2 ov),233/10 (74.2 ov) | 146/10 (47.0 ov)
2,100238,1st T20I,INDIA,BANGLADESH,2024-10-06 19:00:00,2024-10-06 19:00:00,India won by 7 wkts,complete,Shrimant Madhavrao Scindia Cricket Stadium,"Bangladesh tour of India, 2024",T20,132/3 (11.5 ov),127/10 (19.5 ov)
3,100247,2nd T20I,INDIA,BANGLADESH,2024-10-09 19:00:00,2024-10-09 19:00:00,India won by 86 runs,complete,Arun Jaitley Stadium,"Bangladesh tour of India, 2024",T20,221/9 (20.0 ov),135/9 (20.0 ov)
4,100256,3rd T20I,INDIA,BANGLADESH,2024-10-12 19:00:00,2024-10-12 19:00:00,India won by 133 runs,complete,Rajiv Gandhi International Stadium,"Bangladesh tour of India, 2024",T20,297/6 (20.0 ov),164/7 (20.0 ov)
5,56955,1st ODI,BANGLADESH,INDIA,2022-12-04 11:30:00,2022-12-04 11:30:00,Bangladesh won by 1 wkt,complete,Shere Bangla National Stadium,"India tour of Bangladesh, 2022",ODI,187/9 (46.0 ov),186/10 (41.2 ov)
6,56962,2nd ODI,BANGLADESH,INDIA,2022-12-07 11:30:00,2022-12-07 11:30:00,Bangladesh won by 5 runs,complete,Shere Bangla National Stadium,"India tour of Bangladesh, 2022",ODI,271/7 (50.0 ov),266/9 (50.0 ov)
7,56964,3rd ODI,BANGLADESH,INDIA,2022-12-10 11:30:00,2022-12-10 11:30:00,India won by 227 runs,complete,Bir Sreshtho Flight Lieutenant Matiur Rahman S...,"India tour of Bangladesh, 2022",ODI,182/10 (34.0 ov),409/8 (50.0 ov)
8,56969,1st Test,BANGLADESH,INDIA,2022-12-14 09:00:00,2022-12-18 09:00:00,India won by 188 runs,complete,Bir Sreshtho Flight Lieutenant Matiur Rahman S...,"India tour of Bangladesh, 2022",TEST,150/10 (55.5 ov) | 324/10 (113.2 ov),404/10 (133.5 ov) | 258/2 (61.4 ov)
9,56976,2nd Test,BANGLADESH,INDIA,2022-12-22 09:00:00,2022-12-26 09:00:00,India won by 3 wkts,complete,Shere Bangla National Stadium,"India tour of Bangladesh, 2022",TEST,227/10 (73.5 ov) | 231/10 (70.2 ov),314/10 (86.3 ov) | 145/7 (47.0 ov)


In [7]:
def create_series_matches_table(engine):
    """
    Creates 'series_matches' table in MySQL if it doesn't already exist.
    Schema matches the DataFrame returned by extract_series_matches().
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS series_matches (
        match_id BIGINT PRIMARY KEY,
        match_desc VARCHAR(100),
        team1 VARCHAR(50),
        team2 VARCHAR(50),
        start_date DATETIME,
        end_date DATETIME,
        status VARCHAR(100),
        state VARCHAR(50),
        venue VARCHAR(100),
        series_name VARCHAR(150),
        match_format VARCHAR(20),
        team1_score VARCHAR(50),
        team2_score VARCHAR(50)
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'series_matches' is ready.")

engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")

create_series_matches_table(engine)

insert_df_to_mysql(all_matches_df, engine, table_name="series_matches")

Table 'series_matches' is ready.
Data inserted successfully into 'series_matches'!


In [ ]:
import requests
import pandas as pd

def fetch_top_odi_runs(api_key):
    """
    Fetches top ODI run scorers from Cricbuzz API and returns a DataFrame.
    
    Parameters:
        api_key (str): Your RapidAPI key.
        
    Returns:
        pd.DataFrame: DataFrame containing top ODI run scorers.
    """
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"
    querystring = {"statsType": "mostRuns", "matchType": "2"}  # "2" = ODI
    
    headers = {
        "x-rapidapi-key": 'fa8e2f141fmsh93e60c107630b1ep15131djsnd985ef01f9ec',
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    
    # Fetch data from API
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    
    # Extract top ODI runs into a list of dicts
    rows = []
    values = data.get("values", [])
    
    for item in values:
        row = item.get("values", [])
        if len(row) >= 6:
            try:
                rows.append({
                    "player_id": int(row[0]),
                    "player_name": row[1],
                    "matches": int(row[2]),
                    "innings": int(row[3]),
                    "runs": int(row[4]),
                    "average": float(row[5])
                })
            except ValueError:
                continue
    
    return pd.DataFrame(rows)

# --- CALL FUNCTION ---
  # Replace with your key
df_odi_topruns = fetch_top_odi_runs(api_key)

print("Top ODI Run Scorers DataFrame:")
df_odi_topruns


Top ODI Run Scorers DataFrame:


,player_id,player_name,matches,innings,runs,average
0,25,Tendulkar,463,452,18426,44.83
1,104,Sangakkara,404,380,14234,41.99
2,1413,Kohli,302,290,14181,57.88
3,38,R Ponting,375,365,13704,42.04
4,102,S Jayasuriya,445,433,13430,32.36
5,101,Mahela,448,418,12650,33.38
6,35,Inzamam-ul-Haq,378,350,11739,39.53
7,213,Kallis,328,314,11579,44.36
8,29,S Ganguly,311,300,11363,41.02
9,576,Rohit,273,265,11168,48.77


In [19]:
def create_top_odi_runs_table(engine):
    create_query = """
    CREATE TABLE IF NOT EXISTS top_odi_runs (
        player_id BIGINT PRIMARY KEY,
        player_name VARCHAR(100),
        matches INT,
        innings INT,
        runs INT,
        average FLOAT
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'top_odi_runs' is ready.")

create_top_odi_runs_table(engine)

insert_df_to_mysql(df_odi_topruns, engine, table_name="top_odi_runs")

Table 'top_odi_runs' is ready.
Data inserted successfully into 'top_odi_runs'!


In [66]:
import pandas as pd

# Function to extract venue details using your fetch_api function
def extract_venue_details(venue_ids):
    """
    Takes a list of venue IDs, fetches data from the API for each ID,
    and returns a DataFrame with venue_id, venue_name, city, country, capacity.
    If API response is empty, still returns an empty row for that ID.
    """
    all_venues = []

    for vid in venue_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{vid}"
        try:
            data = fetch_api(url)  
        except Exception as e:
            print(f" Error fetching venue {vid}: {e}")
            data = {}

        # Always append a row, even if data is empty
        venue_info = {
            "venue_id": vid,
            "venue_name": data.get("ground", ""),
            "city": data.get("city", ""),
            "country": data.get("country", ""),
            "capacity": data.get("capacity", "")
        }
        all_venues.append(venue_info)

    return pd.DataFrame(all_venues)


# Example usage
venue_ids = [41, 34, 27, 31, 81, 19, 10, 153, 50, 40, 80, 485, 87, 11]
df_venues = extract_venue_details(venue_ids)

print("Venue Details DataFrame:")
df_venues

Venue Details DataFrame:


,venue_id,venue_name,city,country,capacity
0,41,Eden Park,Auckland,New Zealand,"41,000"
1,34,Sydney Cricket Ground,Sydney,Australia,"48,000"
2,27,M.Chinnaswamy Stadium,Bengaluru,India,"40,000"
3,31,Eden Gardens,Kolkata,India,63000
4,81,Wankhede Stadium,Mumbai,India,"33,000"
5,19,Lord's,London,England,30000
6,10,Brabourne Stadium,Mumbai,India,"20,000"
7,153,Dubai International Cricket Stadium,Dubai,United Arab Emirates,"25,000"
8,50,Narendra Modi Stadium,Ahmedabad,India,"132,000"
9,40,Adelaide Oval,Adelaide,Australia,"53,583 (including standing room)"


In [12]:
from sqlalchemy import text

# Create table for venue details
def create_venues_table(engine):
    """
    Creates 'venues' table in MySQL if it doesn't already exist.
    Schema matches the DataFrame returned by extract_venue_details().
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS venues (
        venue_id BIGINT PRIMARY KEY,
        venue_name VARCHAR(150),
        city VARCHAR(100),
        country VARCHAR(100),
        capacity VARCHAR(50)
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'venues' is ready.")


# Create table
create_venues_table(engine)

# Insert DataFrame into MySQL
insert_df_to_mysql(df_venues, engine, table_name="venues")

Table 'venues' is ready.
Data inserted successfully into 'venues'!


In [65]:
import pandas as pd

def extract_players_stats(player_ids):
    """
    Fetch player profile and batting stats for a list of player IDs.
    Returns a DataFrame with:
    player_id, player_name, role, test_runs, odi_runs, t20_runs
    """
    all_players = []

    for pid in player_ids:
        # --- Fetch profile ---
        profile_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}"
        profile_data = fetch_api(profile_url)
        
        player_name = profile_data.get("name")
        role = profile_data.get("role")
        
        # --- Fetch batting stats ---
        batting_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}/batting"
        batting_data = fetch_api(batting_url)

        # Initialize runs
        test_runs = odi_runs = t20_runs = 0

        # Extract runs from 'values' list
        values = batting_data.get("values", [])
        for item in values:
            row = item.get("values", [])
            if row and row[0] == "Runs":
                test_runs = int(row[1]) if row[1].isdigit() else 0
                odi_runs = int(row[2]) if row[2].isdigit() else 0
                t20_runs = int(row[3]) if row[3].isdigit() else 0

        all_players.append({
            "player_id": pid,
            "player_name": player_name,
            "role": role,
            "test_runs": test_runs,
            "odi_runs": odi_runs,
            "t20_runs": t20_runs
        })

    return pd.DataFrame(all_players)

# --- Example usage ---
player_ids = [576, 1413, 265, 247, 2250, 8019, 6326]  # Add any player IDs
df_players = extract_players_stats(player_ids)
df_players


,player_id,player_name,role,test_runs,odi_runs,t20_runs
0,576,Rohit Sharma,Batsman,4301,11168,4231
1,1413,Virat Kohli,Batsman,9230,14181,4188
2,265,MS Dhoni,WK-Batsman,4876,10773,1617
3,247,Chris Gayle,Batsman,7215,10480,1899
4,2250,Steven Smith,Batsman,10477,5800,1094
5,8019,Joe Root,Batsman,13543,7301,893
6,6326,Kane Williamson,Batsman,9276,7236,2575


In [14]:
from sqlalchemy import create_engine, text
import pandas as pd

def create_players_stats_table(engine):
    create_query = """
    CREATE TABLE IF NOT EXISTS players_stats (
        player_id BIGINT PRIMARY KEY,
        player_name VARCHAR(100),
        role VARCHAR(50),
        test_runs INT,
        odi_runs INT,
        t20_runs INT
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'players_stats' is ready.")

engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_players_stats_table(engine)
insert_df_to_mysql(df_players, engine, "players_stats")

Table 'players_stats' is ready.
Data inserted successfully into 'players_stats'!


In [ ]:
import requests
import pandas as pd
from datetime import datetime

# STEP 1: Fetch API data
def fetch_matches_api(url, headers=None):
    if headers is None:
        headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# STEP 2: Extract combined match info
def extract_combined_match_data(match_ids):
    all_matches = []

    for mid in match_ids:
        try:
            # --- API URLs (defined per match) ---
            url_match = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}"
            url_leanback = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/leanback"

            match_data = fetch_matches_api(url_match)
            leanback_data = fetch_matches_api(url_leanback)

            match_id = match_data.get("matchid")
            series_name = match_data.get("seriesname")
            match_format = match_data.get("matchformat")
            match_date = datetime.fromtimestamp(match_data.get("startdate", 0)/1000).date() if match_data.get("startdate") else None
            venue = match_data.get("venueinfo", {}).get("ground")
            team1 = match_data.get("team1", {}).get("teamname")
            team2 = match_data.get("team2", {}).get("teamname")

            toss_info = leanback_data.get("matchheaders", {}).get("tossresults", {})
            toss_winner = toss_info.get("tosswinnername")
            toss_decision = toss_info.get("decision")

            matchheaders = leanback_data.get("matchheaders", {})
            match_winner = None
            if matchheaders.get("status"):
                match_winner = matchheaders["status"].split(" won")[0]

            match_info = {
                "match_id": match_id,
                "format": match_format,
                "series_name": series_name,
                "match_date": match_date,
                "venue": venue,
                "team1": team1,
                "team2": team2,
                "toss_winner": toss_winner,
                "toss_decision": toss_decision,
                "match_winner": match_winner,
                "win_margin": matchheaders.get("status")
            }

            all_matches.append(match_info)

        except Exception as e:
            print(f"Failed to fetch match {mid}: {e}")

    return pd.DataFrame(all_matches)

# Call for multiple matches (all variables at bottom)
api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"
match_ids = [100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762]   # Add more IDs as needed

df_combined_matches = extract_combined_match_data(match_ids)
df_combined_matches


,match_id,format,series_name,match_date,venue,team1,team2,toss_winner,toss_decision,match_winner,win_margin
0,100283,T20,"England tour of India, 2025",2025-01-22,Eden Gardens,India,England,India,Bowling,India,India won by 7 wkts
1,100290,T20,"England tour of India, 2025",2025-01-25,MA Chidambaram Stadium,India,England,India,Bowling,India,India won by 2 wkts
2,130019,T20,Asia Cup 2025,2025-09-10,Dubai International Cricket Stadium,India,United Arab Emirates,India,Bowling,India,India won by 9 wkts
3,82462,T20,"Australia tour of India, 2023",2023-12-01,Shaheed Veer Narayan Singh International Stadium,India,Australia,Australia,Bowling,India,India won by 20 runs
4,116981,T20,"South Africa tour of Australia, 2025",2025-08-12,Marrara Cricket Ground,Australia,South Africa,Australia,Bowling,South Africa,South Africa won by 53 runs
5,116990,T20,"South Africa tour of Australia, 2025",2025-08-16,Cazaly's Stadium,Australia,South Africa,Australia,Bowling,Australia,Australia won by 2 wkts
6,76465,T20,"Pakistan tour of England, 2024",2024-05-25,Edgbaston,England,Pakistan,Pakistan,Bowling,England,England won by 23 runs
7,105804,T20,"South Africa tour of England, 2025",2025-09-12,Emirates Old Trafford,England,South Africa,South Africa,Bowling,England,England won by 146 runs
8,87626,T20,ICC Mens T20 World Cup 2024,2024-06-09,Nassau County International Cricket Stadium,India,Pakistan,Pakistan,Bowling,India,India won by 6 runs
9,87878,T20,ICC Mens T20 World Cup 2024,2024-06-29,Kensington Oval,South Africa,India,India,Batting,India,India won by 7 runs


In [16]:
# Import SQLAlchemy
from sqlalchemy import create_engine, text

# Function to create table if it does not exist
def create_combined_matches_table(engine):
    """
    Creates 'combined_matches' table in MySQL if it doesn't already exist.
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS combined_matches (
        match_id BIGINT PRIMARY KEY,
        format VARCHAR(20),
        series_name VARCHAR(150),
        match_date DATE,
        venue VARCHAR(100),
        team1 VARCHAR(50),
        team2 VARCHAR(50),
        toss_winner VARCHAR(50),
        toss_decision VARCHAR(20),
        match_winner VARCHAR(50),
        win_margin VARCHAR(100)
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'combined_matches' is ready.")

# Connect, create table, and insert data
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_combined_matches_table(engine)
insert_df_to_mysql(df_combined_matches, engine, table_name="combined_matches")

Table 'combined_matches' is ready.
Data inserted successfully into 'combined_matches'!


In [ ]:
# Extract batting stats
def extract_batting_data(match_ids):
    all_records = []

    for mid in match_ids:
        try:
            # --- API URL per match ---
            url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            data = fetch_matches_api(url)

            # Corrected key from 'scoreCard' to 'scorecard'
            for innings in data.get("scorecard", []):
                # The team and player data are not nested under 'batTeamDetails' or 'batsmenData'
                team_name = innings.get("batteamname")
                batsmen = innings.get("batsman", [])

                for bat in batsmen:
                    all_records.append({
                        "match_id": mid,
                        "innings_no": innings.get("inningsid"),
                        "team": team_name,
                        "player_id": bat.get("id"),
                        "player_name": bat.get("name"),
                        "runs": int(bat.get("runs", 0)),
                        "balls_faced": int(bat.get("balls", 0)),
                        "fours": int(bat.get("fours", 0)),
                        "sixes": int(bat.get("sixes", 0)),
                        "strike_rate": float(bat.get("strkrate", 0)),
                        "dismissal": bat.get("outdec")
                    })
        except Exception as e:
            print(f"Failed to fetch match {mid}: {e}")

    return pd.DataFrame(all_records)

api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"
# Call for multiple matches (all variables at bottom)
match_ids = [75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762] # Using a single match ID for demonstration

df_batting = extract_batting_data(match_ids)
df_batting

,match_id,innings_no,team,player_id,player_name,runs,balls_faced,fours,sixes,strike_rate,dismissal
0,75476,1,Pakistan,12139,Shafique,20,24,3,0,83.33,lbw b Siraj
1,75476,1,Pakistan,8364,Imam-ul-Haq,36,38,6,0,94.74,c Rahul b Hardik Pandya
2,75476,1,Pakistan,8359,Babar Azam,50,58,7,0,86.21,b Siraj
3,75476,1,Pakistan,9468,Rizwan,49,69,7,0,71.01,b Bumrah
4,75476,1,Pakistan,9812,Saud Shakeel,6,10,0,0,60.00,lbw b Kuldeep Yadav
...,...,...,...,...,...,...,...,...,...,...,...
886,105762,4,England,13810,Jamie Smith,44,55,4,2,80.00,not out
887,105762,4,England,6670,Chris Woakes,0,0,0,0,0.00,
888,105762,4,England,11436,Brydon Carse,0,0,0,0,0.00,
889,105762,4,England,12747,Josh Tongue,0,0,0,0,0.00,


In [27]:
# Import SQLAlchemy
from sqlalchemy import create_engine, text
import pandas as pd

# Function to create table if it does not exist
def create_batting_table(engine):
    """
    Creates 'batting_data' table in MySQL if it doesn't already exist.
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS batting_data (
        match_id BIGINT,
        innings_no INT,
        team VARCHAR(100),
        player_id BIGINT,
        player_name VARCHAR(100),
        runs INT,
        balls_faced INT,
        fours INT,
        sixes INT,
        strike_rate FLOAT,
        dismissal VARCHAR(200)
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'batting_data' is ready.")

# Connect, create table, and insert data
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_batting_table(engine)
insert_df_to_mysql(df_batting, engine, table_name="batting_data")


Table 'batting_data' is ready.
Data inserted successfully into 'batting_data'!


In [ ]:
def extract_bowling_data(match_ids):
    all_records = []

    for mid in match_ids:
        try:
            # --- API URL per match ---
            url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"

            data = fetch_matches_api(url)

            # Corrected key from 'scoreCard' to 'scorecard'
            for innings in data.get("scorecard", []):
                team_name = innings.get("batteamname") # Get the batting team name
                
                # The bowlers data is under the 'bowler' key, not 'bowlersData'
                bowlers = innings.get("bowler", [])

                for bowler in bowlers:
                    all_records.append({
                        "match_id": mid, # Use the provided match_id
                        "player_id": bowler.get("id"),
                        "player_name": bowler.get("name"),
                        "team": team_name,
                        "overs": float(bowler.get("overs", 0)),
                        "maidens": int(bowler.get("maidens", 0)),
                        "runs_conceded": int(bowler.get("runs", 0)),
                        "wickets": int(bowler.get("wickets", 0)),
                        "economy_rate": float(bowler.get("economy", 0))
                    })
        except Exception as e:
            print(f"Failed to fetch match {mid}: {e}")

    return pd.DataFrame(all_records)
api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"
# Call for multiple matches (all variables at bottom)
match_ids = [75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762] # Using a single match ID for demonstration

df_bowling = extract_bowling_data(match_ids)
df_bowling

,match_id,player_id,player_name,team,overs,maidens,runs_conceded,wickets,economy_rate
0,75476,9311,Bumrah,Pakistan,7.0,1,19,2,2.7
1,75476,10808,Siraj,Pakistan,8.0,0,50,2,6.2
2,75476,9647,Hardik Pandya,Pakistan,6.0,0,34,2,5.7
3,75476,8292,Kuldeep Yadav,Pakistan,10.0,0,35,2,3.5
4,75476,587,Ravindra Jadeja,Pakistan,9.5,0,38,2,3.9
...,...,...,...,...,...,...,...,...,...
435,105762,9311,Jasprit Bumrah,England,19.0,3,57,0,3.0
436,105762,10808,Mohammed Siraj,England,14.0,1,51,0,3.6
437,105762,587,Ravindra Jadeja,England,24.0,1,104,1,4.3
438,105762,10551,Prasidh Krishna,England,15.0,0,92,2,6.1


In [ ]:
# Import SQLAlchemy
from sqlalchemy import create_engine, text
import pandas as pd

# Function to create bowling table if it does not exist
def create_bowling_table(engine):
    """
    Creates 'bowling_data' table in MySQL if it doesn't already exist.
    """
    create_query = """
    CREATE TABLE IF NOT EXISTS bowling_data (
        match_id BIGINT,
        player_id BIGINT,
        player_name VARCHAR(100),
        team VARCHAR(100),
        overs FLOAT,
        maidens INT,
        runs_conceded INT,
        wickets INT,
        economy_rate FLOAT
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'bowling_data' is ready.")

# Connect, create table, and insert data
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_bowling_table(engine)
insert_df_to_mysql(df_bowling, engine, table_name="bowling_data")


Table 'bowling_data' is ready.
Data inserted successfully into 'bowling_data'!


In [ ]:
import requests
import pandas as pd
import re

# Provided API key
api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"

def fetch_players_api(url, headers=None):
    """Fetches JSON data from the specified URL using a RapidAPI key."""
    if headers is None:
        headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None

def extract_fielding_data(match_ids):
    """Extracts fielding data (catches and stumpings) for multiple matches."""
    all_fielding = []
    
    for mid in match_ids:
        try:
            url_scorecard = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            scorecard_data = fetch_players_api(url_scorecard)

            if not scorecard_data:
                continue

            # Dictionary to map player names to their IDs
            player_name_to_id = {}
            score_cards = scorecard_data.get("scorecard", [])

            # First, build a name-to-ID mapping from all players in the scorecard
            for innings in score_cards:
                batsmen_list = innings.get("batsman", [])
                bowlers_list = innings.get("bowler", [])
                
                for player_info in batsmen_list + bowlers_list:
                    name = player_info.get("name")
                    if name:
                        player_name_to_id[name] = player_info.get("id")

            # Now, track fielding stats using player names
            fielding_stats = {}
            for innings in score_cards:
                batsmen_list = innings.get("batsman", [])
                
                for bat_info in batsmen_list:
                    out_desc = bat_info.get("outdec", "")
                    
                    # Regex to find catches (e.g., 'c Fielder Name')
                    catch_match = re.search(r"c\s+([a-zA-Z\s]+)\s+b\s+", out_desc)
                    if not catch_match:
                        catch_match = re.search(r"c\s+([a-zA-Z\s]+)\)$", out_desc)
                    
                    if catch_match:
                        fielder_name = catch_match.group(1).strip()
                        if fielder_name not in fielding_stats:
                            fielding_stats[fielder_name] = {"catches": 0, "stumpings": 0}
                        fielding_stats[fielder_name]["catches"] += 1
                    
                    # Regex to find stumpings (e.g., 'st Wicketkeeper Name')
                    stump_match = re.search(r"st\s+([a-zA-Z\s]+)\s+b\s+", out_desc)
                    if stump_match:
                        keeper_name = stump_match.group(1).strip()
                        if keeper_name not in fielding_stats:
                            fielding_stats[keeper_name] = {"catches": 0, "stumpings": 0}
                        fielding_stats[keeper_name]["stumpings"] += 1
            
            # Create final records using the name-to-ID mapping
            performance_id = 101
            for player_name, stats in fielding_stats.items():
                player_id = player_name_to_id.get(player_name, None)
                if player_id is not None and (stats["catches"] > 0 or stats["stumpings"] > 0):
                    fielding_record = {
                        "performance_id": performance_id,
                        "match_id": mid,
                        "player_id": player_id,
                        "catches": stats["catches"],
                        "stumpings": stats["stumpings"]
                    }
                    all_fielding.append(fielding_record)
                    performance_id += 1

        except Exception as e:
            print(f"Failed to fetch fielding data for match {mid}: {e}")

    return pd.DataFrame(all_fielding)

# Example usage
match_ids = [75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762]
df_fielding = extract_fielding_data(match_ids)
df_fielding

,performance_id,match_id,player_id,catches,stumpings
0,101,75476,9311,1,0
1,102,75476,11808,1,0
2,103,75476,10799,1,0
3,104,75476,11186,1,0
4,105,75476,8367,1,0
...,...,...,...,...,...
240,108,105762,8683,1,0
241,109,105762,13866,1,0
242,110,105762,12770,2,0
243,111,105762,8733,1,0


In [ ]:
from sqlalchemy import create_engine, text

# Create fielding_data table
def create_fielding_table(engine):
    create_query = """
    CREATE TABLE IF NOT EXISTS fielding_data (
        performance_id BIGINT PRIMARY KEY,
        match_id BIGINT,
        player_id BIGINT,
        catches INT,
        stumpings INT
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'fielding_data' is ready.")

# MAIN EXECUTION
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_fielding_table(engine)
insert_df_to_mysql(df_fielding, engine, "fielding_data")


Table 'fielding_data' is ready.
Data inserted successfully into 'fielding_data'!


In [ ]:
def fetch_batting_multiple_matches(match_ids):
    """
    Fetches batting statistics for multiple matches and returns a DataFrame.
    
    Args:
        match_ids (list): A list of match IDs.
    
    Returns:
        pd.DataFrame: A DataFrame containing batting records.
    """
    records = []

    for match_id in match_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/hscard"
        data = fetch_matches_api(url)

        if not data:
            continue

        # Extract the match date from the top-level 'responselastupdated' key
        match_date = None
        match_date_ms = data.get("responselastupdated")
        if match_date_ms:
            match_date = datetime.fromtimestamp(match_date_ms / 1000).date()

        for innings in data.get("scorecard", []):
            innings_no = innings.get("inningsid")
            team_name = innings.get("batteamname")

            # Correct key: 'batsman' is the correct list
            batsmen_list = innings.get("batsman", [])

            for batsman in batsmen_list:
                record = {
                    "match_id": match_id,
                    "innings_no": innings_no,
                    "player_id": batsman.get("id"),
                    "player_name": batsman.get("name"),
                    "runs": int(batsman.get("runs", 0)),
                    "balls_faced": int(batsman.get("balls", 0)),
                    "strike_rate": float(batsman.get("strkrate", 0)),
                    "date": match_date
                }
                records.append(record)
    
    return pd.DataFrame(records)

api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"
# Your match IDs
match_ids = [75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762] # Using a smaller list for a demonstration

df_all_batting = fetch_batting_multiple_matches(match_ids)
df_all_batting

,match_id,innings_no,player_id,player_name,runs,balls_faced,strike_rate,date
0,75476,1,12139,Shafique,20,24,83.33,1970-01-21
1,75476,1,8364,Imam-ul-Haq,36,38,94.74,1970-01-21
2,75476,1,8359,Babar Azam,50,58,86.21,1970-01-21
3,75476,1,9468,Rizwan,49,69,71.01,1970-01-21
4,75476,1,9812,Saud Shakeel,6,10,60.00,1970-01-21
...,...,...,...,...,...,...,...,...
886,105762,4,13810,Jamie Smith,44,55,80.00,1970-01-21
887,105762,4,6670,Chris Woakes,0,0,0.00,1970-01-21
888,105762,4,11436,Brydon Carse,0,0,0.00,1970-01-21
889,105762,4,12747,Josh Tongue,0,0,0.00,1970-01-21


In [ ]:
from sqlalchemy import create_engine, text

# Create table if not exists
def create_batting_table(engine):
    create_query = """
    CREATE TABLE IF NOT EXISTS batters_batting_data (
        match_id BIGINT,
        innings_no INT,
        player_id BIGINT,
        player_name VARCHAR(100),
        runs INT,
        balls_faced INT,
        strike_rate FLOAT,
        date DATE
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'batters_batting_data' is ready.")

engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_batting_table(engine)
insert_df_to_mysql(df_all_batting, engine, "batters_batting_data")

✅ Table 'batters_batting_data' is ready.
Data inserted successfully into 'batters_batting_data'!


In [ ]:
import requests
import pandas as pd
from datetime import datetime

# ---------------- Fetch API Data ----------------
def fetch_players_api(url, headers=None):
    """Fetches JSON data from the specified URL using a RapidAPI key."""
    if headers is None:
        headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None

# ---------------- Extract Bowling Innings Data ----------------
def extract_bowling_innings_data(match_ids):
    """Extracts bowling data for multiple matches, including venue information."""
    all_bowling_innings = []

    for mid in match_ids:
        try:
            # API URLs
            url_scorecard = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            url_match = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}"

            scorecard_data = fetch_players_api(url_scorecard)
            match_data = fetch_players_api(url_match)

            if not scorecard_data or not match_data:
                continue

            # --- Corrected Venue Extraction ---
            venue_info = match_data.get("venueinfo", {})  # lowercase key
            venue = f"{venue_info.get('ground', '')}, {venue_info.get('city', '')}".strip(', ')
            if not venue:
                venue = venue_info.get('ground', '') or venue_info.get('city', '')

            # --- Scorecard and Bowling Data ---
            match_id = match_data.get("matchid", mid)  # lowercase key
            score_cards = scorecard_data.get("scorecard", [])  # lowercase key
            
            for innings in score_cards:
                innings_no = innings.get("inningsid", 0)  # lowercase key
                bowlers_list = innings.get("bowler", [])

                for bowler_info in bowlers_list:
                    player_id = bowler_info.get("id", 0)
                    player_name = bowler_info.get("name", "")
                    overs = float(bowler_info.get("overs", 0))
                    runs_conceded = int(bowler_info.get("runs", 0))
                    wickets = int(bowler_info.get("wickets", 0))
                    economy_rate = float(bowler_info.get("economy", 0))

                    bowling_record = {
                        "match_id": match_id,
                        "innings_no": innings_no,
                        "player_id": player_id,
                        "player_name": player_name,
                        "overs": overs,
                        "runs_conceded": runs_conceded,
                        "wickets": wickets,
                        "economy_rate": economy_rate,
                        "venue": venue
                    }

                    all_bowling_innings.append(bowling_record)

        except Exception as e:
            print(f"Failed to fetch bowling data for match {mid}: {e}")

    return pd.DataFrame(all_bowling_innings)

# ---------------- Provided API key ----------------
api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"

match_ids = [100283, 38617, 38612, 38627, 75623, 78621, 48121, 70326, 91787, 60035, 100310, 75563]

df_bowling_innings = extract_bowling_innings_data(match_ids)

df_bowling_innings


,match_id,innings_no,player_id,player_name,overs,runs_conceded,wickets,economy_rate,venue
0,100283,1,13217,Arshdeep Singh,4.0,17,2,4.2,"Eden Gardens, Kolkata"
1,100283,1,9647,Hardik Pandya,4.0,42,2,10.5,"Eden Gardens, Kolkata"
2,100283,1,12926,Varun Chakravarthy,4.0,23,3,5.8,"Eden Gardens, Kolkata"
3,100283,1,8808,Axar,4.0,22,2,5.5,"Eden Gardens, Kolkata"
4,100283,1,14659,Ravi Bishnoi,4.0,22,0,5.5,"Eden Gardens, Kolkata"
...,...,...,...,...,...,...,...,...,...
169,75563,2,9311,Jasprit Bumrah,5.0,8,1,1.6,"Wankhede Stadium, Mumbai"
170,75563,2,10808,Mohammed Siraj,7.0,16,3,2.3,"Wankhede Stadium, Mumbai"
171,75563,2,7909,Mohammed Shami,5.0,18,5,3.6,"Wankhede Stadium, Mumbai"
172,75563,2,8292,Kuldeep Yadav,2.0,3,0,1.5,"Wankhede Stadium, Mumbai"


In [ ]:
from sqlalchemy import create_engine, text

# Create table if not exists
def create_bowling_innings_table(engine):
    create_query = """
    CREATE TABLE IF NOT EXISTS bowlers_bowling_venue_data (
        match_id INT,
        innings_no INT,
        player_id INT,
        player_name VARCHAR(100),
        overs FLOAT,
        runs_conceded INT,
        wickets INT,
        economy_rate FLOAT,
        venue VARCHAR(200)
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'bowlers_bowling_venue_data' is ready.")

# Insert DataFrame to MySQL
# Connect to DB
engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
# Create table first
create_bowling_innings_table(engine)
# Insert bowling DataFrame (df_bowling_innings is already created by your fetch function)
insert_df_to_mysql(df_bowling_innings, engine, "bowlers_bowling_venue_data")

Table 'bowlers_bowling_venue_data' is ready.
Data inserted successfully into 'bowlers_bowling_venue_data'!


In [ ]:
def extract_partnerships_data(match_ids):
    """
    Extracts partnership data for multiple matches, including the wicket number.
    """
    all_partnerships = []

    for mid in match_ids:
        try:
            url_scorecard = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            scorecard_data = fetch_players_api(url_scorecard)

            if not scorecard_data:
                continue

            score_cards = scorecard_data.get("scorecard", [])
            
            for innings in score_cards:
                innings_no = innings.get("inningsid", 0)
                
                # Correct key for partnerships
                partnerships_list = innings.get("partnership", {}).get("partnership", [])
                
                # Use enumerate to get the wicket number based on the order in the list
                for index, partnership_info in enumerate(partnerships_list):
                    batter1_id = partnership_info.get("bat1id", 0)
                    batter2_id = partnership_info.get("bat2id", 0)
                    batter1_name = partnership_info.get("bat1name", "")
                    batter2_name = partnership_info.get("bat2name", "")
                    runs_partnership = partnership_info.get("totalruns", 0)
                    balls_faced = partnership_info.get("totalballs", 0)
                    
                    # The wicket_fallen number is the index + 1
                    wicket_fallen = index + 1

                    partnership_record = {
                        "match_id": mid,
                        "innings_no": innings_no,
                        "batter1_id": batter1_id,
                        "batter2_id": batter2_id,
                        "batter1_name": batter1_name,
                        "batter2_name": batter2_name,
                        "runs_partnership": runs_partnership,
                        "balls_faced": balls_faced,
                        "wicket_fallen": wicket_fallen
                    }

                    all_partnerships.append(partnership_record)

        except Exception as e:
            print(f"Failed to fetch partnerships data for match {mid}: {e}")

    return pd.DataFrame(all_partnerships)

api_key = "42ba173510msh8668786dd4440a9p1855c7jsnb61dc18d9c24"

# Call for multiple matches
match_ids = [38321, 50958, 59965, 48205, 48401]
df_partnerships = extract_partnerships_data(match_ids)
df_partnerships

,match_id,innings_no,batter1_id,batter2_id,batter1_name,batter2_name,runs_partnership,balls_faced,wicket_fallen
0,38321,1,6534,6507,Roy,Bairstow,12,9,1
1,38321,1,6534,8019,Roy,Root,0,3,2
2,38321,1,6534,6557,Roy,Stokes,54,47,3
3,38321,1,2258,6557,Buttler,Stokes,8,21,4
4,38321,1,2258,6692,Buttler,Moeen Ali,75,84,5
...,...,...,...,...,...,...,...,...,...
71,48401,2,9427,8271,Hooda,Samson,27,34,5
72,48401,2,9427,8808,Hooda,Axar,51,33,6
73,48401,2,8683,8808,Thakur,Axar,22,12,7
74,48401,2,9781,8808,Avesh Khan,Axar,24,19,8


In [ ]:
from sqlalchemy import create_engine, text

def create_partnerships_table(engine):
    create_query = """
    CREATE TABLE IF NOT EXISTS players_partnerships_data (
        match_id INT,
        innings_no INT,
        batter1_id INT,
        batter2_id INT,
        batter1_name VARCHAR(100),
        batter2_name VARCHAR(100),
        runs_partnership INT,
        balls_faced INT,
        wicket_fallen INT
    )
    """
    with engine.connect() as conn:
        conn.execute(text(create_query))
        print("Table 'players_partnerships_data' is ready.")

engine = create_mysql_engine("root", "root", "localhost", "cricbuzz_db")
create_partnerships_table(engine)
insert_df_to_mysql(df_partnerships, engine, "players_partnerships_data")

Table 'players_partnerships_data' is ready.
Data inserted successfully into 'players_partnerships_data'!
